# Import Dependencies.

In [16]:
import azureml
import joblib
import pickle
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

experiment_name = 'Mushroom'
amlcompute_cluster_name = "comp"
ds_key = "Mushroom"
ds_description_text = "This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms"
project_folder = './mushroom-project'

SDK version: 1.20.0


# Workspace
+ copy config json to folder
+ create workspace from config
+ create compute cluster 

In [9]:
ws = Workspace.from_config()
experiment=Experiment(ws, experiment_name)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
run = experiment.start_logging()

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().


quick-starts-ws-137304
aml-quickstarts-137304
southcentralus
2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Dataset

TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

In [10]:
found = False
if ds_key in ws.datasets.keys():
        found = True
        dataset = ws.datasets[ds_key]

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/mixmasteru/MLEND-capstone/main/data/mushrooms.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=ds_key,
                                   description=ds_description_text)
df = dataset.to_pandas_dataframe()
df.describe()
dataset.take(5).to_pandas_dataframe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,True,p,False,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,True,a,False,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,True,l,False,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,True,p,False,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,False,n,False,w,b,k,...,s,w,w,p,w,o,e,n,a,g


# Cleanup

In [ ]:
from train import clean_data
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory

# Use the clean_data function to clean your data.
x, y = clean_data(df)
x["class"] = y
x_train, x_test = train_test_split(x, train_size=0.75, test_size=0.25, random_state=101)


Path("./input").mkdir(parents=True, exist_ok=True)
pd.DataFrame(x_train).to_csv("./input/x_train.csv", index=False)
pd.DataFrame(x_train).to_csv("./input/x_test.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./input', target_path='input', overwrite=True, show_progress=True)

x_train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('input/x_train.csv'))
x_test_data = TabularDatasetFactory.from_delimited_files(path=ds.path('input/x_test.csv'))


# AutoML
 w

In [12]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "featurization": 'auto'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=x_train_data,
                             validation_data=x_test_data,
                             label_column_name="class",
                             path = project_folder,
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

Running on remote.

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more abou

{'runId': 'AutoML_909fe44b-6ad7-4d0b-8d0f-53b26d017a12',
 'target': 'comp',
 'status': 'Completed',
 'startTimeUtc': '2021-02-05T10:40:20.611784Z',
 'endTimeUtc': '2021-02-05T11:05:10.743804Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'comp',
  'AMLSettingsJsonString': '{"path":null,"name":"Mushroom","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-137304","workspace_name":"quick-starts-ws-137304","region":"southcentralus","compute_target":"comp","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featurization":

# Best Model

In [17]:
# Retrieve best model from Run
best_run, best_model = remote_run.get_output()
print(best_run)
print(best_run.get_metrics())
joblib.dump(best_model, 'outputs/automl_model.joblib')


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: Mushroom,
Id: AutoML_909fe44b-6ad7-4d0b-8d0f-53b26d017a12_1,
Type: azureml.scriptrun,
Status: Completed)
{'recall_score_macro': 1.0, 'average_precision_score_macro': 1.0, 'AUC_micro': 1.0, 'recall_score_weighted': 1.0, 'balanced_accuracy': 1.0, 'f1_score_macro': 1.0, 'accuracy': 1.0, 'AUC_macro': 1.0, 'f1_score_weighted': 1.0, 'log_loss': 0.00169919847277639, 'norm_macro_recall': 1.0, 'precision_score_weighted': 1.0, 'average_precision_score_weighted': 1.0, 'average_precision_score_micro': 1.0, 'precision_score_micro': 1.0, 'f1_score_micro': 1.0, 'precision_score_macro': 1.0, 'recall_score_micro': 1.0, 'AUC_weighted': 1.0, 'matthews_correlation': 1.0, 'weighted_accuracy': 1.0, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_909fe44b-6ad7-4d0b-8d0f-53b26d017a12_1/accuracy_table', 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_909fe44b-6ad7-4d0b-8d0f-53b26d017a12_1/confusion_matrix'}


['outputs/automl_model.joblib']

# Hyperdrive

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
             "C": uniform(0.1, 2.0),
             "max_iter": choice(50, 100, 250)
         }
     )

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")


# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',
              entry_script='train.py',
              compute_target=compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=30,
                                     max_concurrent_runs=4)

# hyperdrive_run

In [ ]:
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

# hyperdrive_run

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_metrics())
best_run.get_file_names()
best_run.download_file(name="outputs/hyperdrive_model.joblib", output_file_path="./outputs/")